# 第 3 章：编码注意力机制

本章节使用的软件包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.5.1


## 3.1 长序列模型的问题

- 由于源语言和目标语言的语法结构存在差异，逐字翻译文本是不可行的，在长文本序列中会丢失历史信息
- 在引入 Transformer 模型之前，编码器-解码器 RNN 通常用于机器翻译任务
- 在此设置中，编码器处理来自源语言的标记序列，使用隐藏状态（神经网络中的一种中间层）来生成整个输入序列的压缩表示

## 3.2 使用注意机制捕获数据依赖关系

- 通过注意力机制，网络的文本生成解码器部分能够有选择地访问所有输入标记，这意味着在生成特定输出标记时，某些输入标记比其他输入标记更重要
- transformer 中的自注意力是一种旨在增强输入表示的技术，它使序列中的每个位置能够与同一序列中其他每个位置互动并确定它们的相关性

## 3.3 使用自注意力机制关注输入的不同部分

### 3.3.1 一种没有可训练权重的简单自注意力机制

- 本节解释了一种非常简化的自注意力变体，它不包含任何可训练的权重
- 这纯粹是为了说明目的，而不是 Transformer 中使用的注意力机制
- 下一节，即第 3.3.2 节，将扩展这种简单的注意力机制以实现真正的自注意力机制
- 假设我们得到一个输入序列 $x^{(1)}$ 到 $x^{(T)}$
- 输入是一个文本（例如，像“你的旅程从一步开始”这样的句子），它已经按照第 2 章中的描述转换为 token 嵌入
- 例如，$x^{(1)}$ 是一个 d 维向量，表示单词“你的”，依此类推
- **目标**：为 $x^{(1)}$ 到 $x^{(T)}$ 中的每个输入序列元素 $x^{(i)}$ 计算上下文向量 $z^{(i)}$（其中 $z$ 和 $x$ 具有相同的维度）
- 上下文向量$z^{(i)}$ 是输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权和
- 上下文向量是特定于某个输入的“上下文”
- 我们不再将 $x^{(i)}$ 作为任意输入标记的占位符，而是考虑第二个输入 $x^{(2)}$
- 继续举一个具体的例子，我们不再将占位符 $z^{(i)}$ 作为第二个输出上下文向量，而是考虑第二个输出上下文向量 $z^{(2)}$
- 第二个上下文向量 $z^{(2)}$ 是所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权和，相对于第二个输入元素 $x^{(2)}$ 加权
- 注意力权重是确定每个输入元素在计算 $z^{(2)}$ 时对加权和贡献多少的权重
- 简而言之，考虑$z^{(2)}$ 是 $x^{(2)}$ 的修改版本，它还包含与当前给定任务相关的所有其他输入元素的信息

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

-（请注意，此图中的数字被截断为小数点后一位
数字，以减少视觉混乱；同样，其他图也可能包含截断值）

- 按照惯例，未标准化的注意力权重被称为**注意力分数 **，而标准化的注意力分数（总和为 1）被称为**注意力权重**

- 以下代码逐步演示了上图

<br>

- **步骤 1：**计算非规范化注意力得分 $\omega$
- 假设我们使用第二个输入标记作为查询，即 $q^{(2)} = x^{(2)}$，我们通过点积计算非规范化注意力得分：
- $\omega_{21} = x^{(1)} q^{(2)\top}$
- $\omega_{22} = x^{(2)} q^{(2)\top}$
- $\omega_{23} = x^{(3)} q^{(2)\top}$
- ...
- $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 上图中，$\omega$ 是用于表示非规范化注意力得分的希腊字母“omega”
- 下标$\omega_{21}$ 中的“21”表示输入序列元素 2 被用作对输入序列元素 1 的查询

- 假设我们有以下输入句子，该句子已经嵌入到第 3 章中描述的三维向量中（为了便于说明，我们在这里使用非常小的嵌入维度，以便它能够放入页面上而无换行符）：

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

-（在本书中，我们遵循常见的机器学习和深度学习惯例，其中训练示例表示为行，特征值表示为列；就上图所示的张量而言，每行代表一个单词，每列代表一个嵌入维度）

- 本节的主要目标是演示如何使用第二个输入序列 $x^{(2)}$ 作为查询来计算上下文向量 $z^{(2)}$

- 图中描绘了此过程的初始步骤，其中包括通过点积运算计算 $x^{(2)}$
和所有其他输入元素之间的注意力分数 ω

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 我们使用输入序列元素 2，$x^{(2)}$ 作为示例来计算上下文向量 $z^{(2)}$；在本节后面，我们将推广此方法以计算所有上下文向量。

- **步骤 1：** 通过计算查询 $x^{(2)}$ 和所有其他输入标记之间的点积来计算非规范化注意力分数：

In [ ]:
query = inputs[1]  # 2nd input token is the query: [0.55, 0.87, 0.66]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 附注：点积本质上是将两个向量按元素相乘并对所得乘积求和的简写：

In [7]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **步骤 2：** 将未标准化的注意力得分 ("omegas", $\omega$) 标准化，使它们的总和为 1
- 这是一个将未标准化的注意力得分标准化为总和为 1 的简单方法（一种惯例，对解释有用，对训练稳定性很重要）：

In [8]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实践中，使用 softmax 函数进行归一化是常见的，也是推荐的，因为它更善于处理极值，并且在训练期间具有更理想的梯度特性。
- 这是一个用于缩放的 softmax 函数的简单实现，它还将向量元素归一化，使它们的总和为 1：

In [9]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 由于溢出和下溢问题，上述简单实现可能会因大或小输入值而遭受数值不稳定问题
- 因此，在实践中，建议改用 softmax 的 PyTorch 实现，该实现已针对性能进行了高度优化：

In [10]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤 3**：通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘来计算上下文向量 $z^{(2)}$，并对得到的向量求和：

In [20]:
query = inputs[1] # 2nd input token is the query: [0.55, 0.87, 0.66]
print("attn_weights_2: ", attn_weights_2)
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print("context_vec_2: ", context_vec_2)

attn_weights_2:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
context_vec_2:  tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有输入标记的注意力权重

#### 推广到所有输入序列标记：

- 上面，我们计算了输入 2 的注意力权重和上下文向量（如下图突出显示的行所示）


- 将前面的**步骤 1**应用于所有成对元素，以计算非标准化注意力分数矩阵：

In [21]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 我们可以通过矩阵乘法更有效地实现上述目标：

In [22]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与之前的**步骤 2**类似，我们对每一行进行规范化，使得每行的值总和为 1：

In [23]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每行的值确实总和为 1：

In [24]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用前面的**步骤 3** 来计算所有上下文向量：

In [25]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 作为健全性检查，先前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上面的第二行找到：

In [26]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 使用可训练权重实现自注意力

### 3.4.1 逐步计算注意力权重

- 在本节中，我们将实现原始 Transformer 架构、GPT 模型和大多数其他流行 LLM 中使用的自注意力机制
- 这种自注意力机制也称为“缩放点积注意力”
- 总体思路与之前类似：
  - 我们希望将上下文向量计算为特定于某个输入元素的输入向量的加权和
  - 对于上述内容，我们需要注意权重
- 如您所见，与之前介绍的基本注意力机制相比，只有细微的差别：
  - 最显着的区别是引入了在模型训练期间更新的权重矩阵
  - 这些可训练的权重矩阵至关重要，以便模型（具体而言，模型内部的注意力模块）可以学习生成“良好”的上下文向量

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 逐步实现自注意力机制，我们首先引入三个训练权重矩阵 $W_q$、$W_k$ 和 $W_v$
- 这三个矩阵用于通过矩阵乘法将嵌入的输入标记 $x^{(i)}$ 投影到查询、键和值向量中：

  - 查询向量：$q^{(i)} = W_q \,x^{(i)}$
  - 键向量：$k^{(i)} = W_k \,x^{(i)}$
  - 值向量：$v^{(i)} = W_v \,x^{(i)}$
- 输入 $x$ 和查询向量 $q$ 的嵌入维度可以相同或不同，具体取决于模型的设计和具体实现
- 在 GPT 模型中，输入和输出维度通常相同，但为了便于说明，为了更好地跟踪计算，我们在这里选择不同的输入和输出维度：

In [27]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- 下面，我们初始化三个权重矩阵；请注意，为了便于说明，我们设置了 `requires_grad=False` 以减少输出中的混乱，但如果我们要使用权重矩阵进行模型训练，我们将设置 `requires_grad=True` 以在模型训练期间更新这些矩阵

In [28]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 接下来我们计算查询、键和值向量：

In [29]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 如下所示，我们成功地将 6 个输入标记从 3D 投影到 2D 嵌入空间：

In [31]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步，**步骤 2**中，我们通过计算查询和每个关键向量之间的点积来计算非标准化注意力分数：

In [32]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 由于我们有 6 个输入，因此对于给定的查询向量，我们有 6 个注意力分数：

In [33]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


- 接下来，在**步骤 3**中，我们使用之前使用的 softmax 函数计算注意力权重（总和为 1 的标准化注意力分数）
- 与之前的不同之处在于，我们现在通过将注意力分数除以嵌入维度的平方根 $\sqrt{d_k}$（即 `d_k**0.5`）来缩放注意力分数：

In [34]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


- 在**步骤 4** 中，我们现在计算输入查询向量 2 的上下文向量：

In [35]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 实现一个紧凑的 SelfAttention 类

- 综上所述，我们可以实现自注意力机制，如下所示：

In [36]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- 我们可以使用 PyTorch 的线性层简化上述实现，如果我们禁用偏差单元，这些层相当于矩阵乘法
- 与手动“nn.Parameter(torch.rand(...)”方法相比，使用“nn.Linear”的另一大优势是“nn.Linear”具有首选的权重初始化方案，从而可以实现更稳定的模型训练

In [37]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 请注意，“SelfAttention_v1”和“SelfAttention_v2”给出不同的输出，因为它们对权重矩阵使用不同的初始权重

## 3.5 用因果注意力隐藏未来的单词

- 在因果注意力中，对角线上方的注意力权重被掩盖，确保对于任何给定的输入，LLM 在使用注意力权重计算上下文向量时无法利用未来的标记

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 应用因果注意力掩码

- 在本节中，我们将之前的自注意力机制转换为因果自注意力机制
- 因果自注意力确保模型对序列中某个位置的预测仅依赖于先前位置的已知输出，而不依赖于未来位置
- 简而言之，这确保每个下一个单词的预测仅依赖于前面的单词
- 为了实现这一点，对于每个给定的标记，我们屏蔽掉未来的标记（输入文本中当前标记之后的标记）

- 为了说明和实现因果自注意力，让我们使用上一节中的注意力分数和权重：

In [38]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 掩盖未来注意力权重的最简单方法是通过 PyTorch 的 tril 函数创建一个掩码，将主对角线下方的元素（包括对角线本身）设置为 1，将主对角线上方的元素设置为 0：

In [39]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与这个掩码相乘，以将对角线上方的注意力分数归零：

In [40]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 但是，如果像上面一样在 softmax 之后应用掩码，则会破坏 softmax 创建的概率分布
- Softmax 确保所有输出值的总和为 1
- 在 softmax 之后进行掩码需要重新规范化输出以使其总和再次为 1，这会使过程变得复杂，并可能导致意想不到的效果

- 为了确保行总和为 1，我们可以按如下方式对注意力权重进行标准化：

In [41]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 虽然我们现在在技术上已经完成了因果注意力机制的编码，但让我们简单看一下实现上述相同目标的更有效方法
- 因此，我们可以在对角线上方未归一化的注意力分数进入 softmax 函数之前，用负无穷大掩盖它们，而不是将对角线上方的注意力权重清零并重新归一化结果：

In [42]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 正如我们下面看到的，现在每行的注意力权重再次正确地加起来为 1：

In [43]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 使用 dropout 掩盖额外的注意力权重

- 此外，我们还在训练过程中应用了 dropout 来减少过拟合
- Dropout 可以应用于多个地方：
- 例如，在计算注意力权重之后；
- 或者在将注意力权重与值向量相乘之后
- 在这里，我们将在计算注意力权重后应用 dropout mask，因为它更常见

- 此外，在这个特定示例中，我们使用 50% 的 dropout 率，这意味着随机屏蔽掉一半的注意力权重。（当我们稍后训练 GPT 模型时，我们将使用较低的 dropout 率，例如 0.1 或 0.2

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 如果我们采用 0.5（50%）的丢弃率，则未丢弃的值将相应按 1/0.5 = 2 的倍数缩放
- 缩放比例通过公式 1 / (1 - `dropout_rate`) 计算

In [57]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [62]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


### 3.5.3 实现紧凑因果自注意力类

- 现在，我们准备实现自注意力的工作实现，包括因果和 dropout 掩码
- 还有一件事是实现代码来处理由多个输入组成的批次，以便我们的 `CausalAttention` 类支持我们在第 2 章中实现的数据加载器生成的批次输出
- 为简单起见，为了模拟这样的批次输入，我们复制输入文本示例：

In [63]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [64]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


## 3.6 将单头注意力扩展到多头注意力

### 3.6.1 堆叠多个单头注意力层

- 以下是之前实现的自注意力的总结（为简单起见，未显示因果和 dropout 掩码）

- 这也称为单头注意力：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 我们只需堆叠多个单头注意力模块即可获得多头注意力模块：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 多头注意力背后的主要思想是使用不同的、学习到的线性投影多次（并行）运行注意力机制。这允许模型共同关注来自不同位置的不同表示子空间的信息。

In [65]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上述实现中，嵌入维度为 4，因为我们将 `d_out=2` 作为键、查询和值向量以及上下文向量的嵌入维度。由于我们有 2 个注意力头，因此输出嵌入维度为 2*2=4

### 3.6.2 通过权重分割实现多头注意力

- 虽然以上是多头注意力的直观且功能齐全的实现（包装了之前的单头注意力`CausalAttention`实现），但我们可以编写一个名为`MultiHeadAttention`的独立类来实现相同的功能

- 我们不为这个独立的`MultiHeadAttention`类连接单个注意力头
- 相反，我们创建单个W_query、W_key和W_value权重矩阵，然后将它们拆分为每个注意力头的单独矩阵：

In [66]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，以上内容本质上是 `MultiHeadAttentionWrapper` 的重写版本，效率更高
- 由于随机权重初始化不同，因此生成的输出看起来有些不同，但两者都是功能齐全的实现，可用于我们将在后续章节中实现的 GPT 类
- 请注意，此外，我们还向上面的 `MultiHeadAttention` 类添加了一个线性投影层（`self.out_proj`）。这只是一个不会改变维度的线性变换。在 LLM 实现中使用这样的投影层是一种标准惯例，但并非绝对必要（最近的研究表明，可以在不影响建模性能的情况下将其删除；请参阅本章末尾的进一步阅读部分）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 请注意，如果您对上述内容的紧凑高效实现感兴趣，您还可以考虑 PyTorch 中的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类

- 由于上述实现乍一看可能有点复杂，让我们看看执行 `attn_scores = queries @ keys.transpose(2, 3)` 时会发生什么：

In [67]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现将处理 4 维输入张量，以便在最后 2 个维度（num_tokens、head_dim）之间进行矩阵乘法，然后对各个头部重复此操作

- 例如，以下成为一种更紧凑的方式来分别计算每个头部的矩阵乘法：

In [68]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])
